**stage2_DANN**. This notebook is the initial attempt to train a DANN with UMAFall_waist as source data and UPFall_waist as target data.

**Edit**<br/>

**TODO**<br/>

# Import packages and get authenticated

In [1]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', 500)
from tqdm import tqdm_notebook as tqdm
from IPython.display import display
import os
import sys
sys.path.append('/content/drive/My Drive/中研院/repo/')

from utilities import *
from models import *
from dataset_util import *
from training_util import *
from eval_util import *

import time
import datetime
from datetime import datetime
import json

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc( 'savefig', facecolor = 'white' )

from sklearn.decomposition import PCA

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

### model unit testing

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print('show GPU device name:', torch.cuda.get_device_name(0))
model_1 = FeatureExtractor(input_dim=66).to(device).float()

test_input = torch.randn((8, 3, 66), dtype=torch.double)

test_input = test_input.to(device)
    # labels = labels.to(device).long()

feature_out = model_1(test_input)
print('show model_1 output size:', feature_out.size())

feature_out.data.detach().cpu().numpy()

feature_out_dim =  feature_out.size()[1]
model_2 = ClassClassifier(num_classes=2, input_dim=feature_out_dim).to(device).float()
model_3 = DomainClassifier(num_classes=2, input_dim=feature_out_dim).to(device).float()

model_2_out = model_2(feature_out)
print('show model_2 output size:', model_2_out.size())

model_3_out = model_3(feature_out, 1)
print('show model_3 output size:', model_3_out.size())

model_4 = CascadedModel(model_1, model_2)
# model_4 = nn.Sequential(model_1, model_2)

print('model_4 output size', model_4(test_input).size())

dann = DannModel(device, class_N=2, domain_N=2, channel_n=5, input_dim=66).to(device).float()
feature_out, class_output, domain_output = dann(test_input)
print('dann output size', feature_out.size(), class_output.size(), domain_output.size())



show GPU device name: Tesla K80
FeatureExtractor_total_params: 1824
show model_1 output size: torch.Size([8, 576])
show model_2 output size: torch.Size([8, 2])
show model_3 output size: torch.Size([8, 2])
model_4 output size torch.Size([8, 2])
FeatureExtractor_total_params: 240
DannModel_total_params: 964
dann output size torch.Size([8, 180]) torch.Size([8, 2]) torch.Size([8, 2])


# Exporting function for model performance, notebook parameters, and model

### performance_table

In [0]:
# def performance_table(df_performance_table, src_name, tgt_name, training_params, inputdir, outputdir):
#   task_name = src_name+'_'+tgt_name
#   # df_performance_table[task_name] = ''

#   start_time = time.time()
#   # print('========================transferring knowledge from source({}) to target({})========================'.format(src_name, tgt_name))
#   print('\n==========================================================================================================================')
#   print('======================  train on source, val on target(source={} to target={})  ======================'.format(src_name, tgt_name))
#   print('==========================================================================================================================\n')
#   source_outputs = BaselineModel_fitting(training_params, src_name, tgt_name, inputdir, outputdir+'source/')
#   print('\n==========================================================================================================================')
#   print('======================  train on target, val on target(source={} to target={})  ======================'.format(src_name, tgt_name))
#   print('==========================================================================================================================\n')
#   target_outputs = BaselineModel_fitting(training_params, tgt_name, src_name, inputdir, outputdir+'target/')
#   print('\n==========================================================================================================================')
#   print('======================  DANN training transferring knowledge(source={} to target={})  ======================'.format(src_name, tgt_name))
#   print('==========================================================================================================================\n')
  
#   # print('========================DANN training transferring knowledge from source({}) to target({})========================'.format(src_name, tgt_name))
#   dann_outputs = DannModel_fitting(training_params, src_name, tgt_name, inputdir, outputdir+'dann/')

#   elapsed_time = time.time() - start_time
#   print('time elapsed:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

#   (val_tgt_class_acc_mean, val_tgt_class_acc_std), (val_domain_acc_mean, val_domain_acc_std) = dann_outputs
#   (_,_), (source_tgt_acc_mean, source_tgt_acc_std) = source_outputs
#   (target_tgt_acc_mean, target_tgt_acc_std), (_,_) = target_outputs

#   df_performance_table.loc['source',task_name] = '{:.3f}±{:.3f}'.format(source_tgt_acc_mean, source_tgt_acc_std)
#   df_performance_table.loc['DANN',task_name] = '{:.3f}±{:.3f}'.format(val_tgt_class_acc_mean, val_tgt_class_acc_std)
#   df_performance_table.loc['target',task_name] = '{:.3f}±{:.3f}'.format(target_tgt_acc_mean, target_tgt_acc_std)
#   df_performance_table.loc['domain',task_name] = '{:.3f}±{:.3f}'.format(val_domain_acc_mean, val_domain_acc_std)

#   return df_performance_table

# ADDA implementation

In [0]:
# class AddaModel(nn.Module):
#   def __init__(self, device, class_N=2, domain_N=2, channel_n=16, input_dim=10):
#     super(AddaModel, self).__init__()
#     self.feature_extractor = FeatureExtractor(input_dim=input_dim, channel_n=channel_n).to(device).float()
#     cnn_layer1_dim = (input_dim+2*2-1*(3-1)-1)+1
#     pool_layer1_dim = (cnn_layer1_dim-1*(2-1)-1)/2+1

#     cnn_layer2_dim = (pool_layer1_dim+2*2-1*(3-1)-1)+1
#     pool_layer2_dim = (cnn_layer2_dim-1*(2-1)-1)/2+1

#     feature_out_dim = int(pool_layer2_dim*channel_n*2)

#     self.class_classfier = ClassClassifier(num_classes=class_N, input_dim=feature_out_dim).to(device).float()
#     self.domain_classifier = DomainClassifier(num_classes=domain_N, input_dim=feature_out_dim).to(device).float()
      
#   def forward(self, x):
#     feature_out = self.feature_extractor(x)
#     class_output = self.class_classfier(feature_out)
#     domain_output = self.domain_classifier(feature_out, 1)
#     return feature_out, class_output, domain_output


In [0]:
# # fine-tuning
# tasks_list = [('UMAFall_waist', 'UPFall_belt')]

# # training_params = {
# #     'classes_n': 2,
# #     'CV_n': 17,
# #     'num_epochs': 10,
# #     'channel_n': 4,
# #     'batch_size': 64,
# #     'learning_rate': 0.01,
# #     'extractor_type': 'CNN'}

# training_params =
#   {
#     'classes_n': 2,
#     'CV_n': 17,
#     'num_epochs': 10,
#     'channel_n': 16,
#     'batch_size': 4,
#     'learning_rate': 0.01,
#     'extractor_type': 'CNN'}

# start_time = time.time()
# df_performance_table = pd.DataFrame('', index=['source', 'DANN', 'target', 'domain'], columns=[])

# for task_item in tasks_list:
#   (src_name, tgt_name) = task_item

#   inputdir = '/content/drive/My Drive/中研院/data_mic/stage1_preprocessed_18hz_LOO/'
#   outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz_LOO/{}_{}/'.format(src_name, tgt_name)
#   if not os.path.exists(outputdir):
#       os.makedirs(outputdir)
#   print('outputdir for stage2 output:', outputdir)

#   df_performance_table = performance_table(df_performance_table, src_name, tgt_name, training_params, inputdir, outputdir)
#   # display(df_performance_table)

# time_elapsed = time.time() - start_time
# print('time elapsed:', time_elapsed)
# df_performance_table.loc['time_elapsed'] = time_elapsed
# display(df_performance_table)

In [0]:
# df_performance_table

# df_outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz/hyperparameter_ft/{}_{}/'.format(src_name, tgt_name)

# if not os.path.exists(df_outputdir):
#   os.makedirs(df_outputdir)
# df_performance_table.to_csv(df_outputdir+'df_performance_table_learning_rate{}.csv'.format(training_params['learning_rate']), encoding='utf-8')


# df_outputdir+'df_performance_table_learning_rate{}.csv'.format(training_params['learning_rate'])

# optimal hyperparameter

In [0]:
# optimal

# tasks_list = [('UMAFall_waist', 'UPFall_belt')]
tasks_list = [('UPFall_belt', 'UMAFall_waist')]
# tasks_list = [('UMAFall_ankle', 'UPFall_ankle')]
# tasks_list = [('UMAFall_wrist', 'UPFall_wrist')]
# tasks_list = [('UMAFall_leg', 'UPFall_rightpocket')]

optimal_training_params = {
  'classes_n': 2,
  'CV_n': 17,
  'num_epochs': 10,
  'channel_n': 4,
  'batch_size': 4,
  'learning_rate': 0.01,
  'extractor_type': 'CNN'}

# for i, training_params in enumerate(training_params_list):
df_performance_table = pd.DataFrame('', index=['source', 'DANN', 'target', 'domain'], columns=[])

for task_item in tasks_list:
  start_time = time.time()

  (src_name, tgt_name) = task_item

  inputdir = '/content/drive/My Drive/中研院/data_mic/stage1_preprocessed_18hz_LOO/'
  outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz_LOO_testing/{}_{}/'.format(src_name, tgt_name)
  if not os.path.exists(outputdir):
      os.makedirs(outputdir)
  print('outputdir for stage2 output:', outputdir)

  df_performance_table = performance_table_v2(df_performance_table, src_name, tgt_name, optimal_training_params, inputdir, outputdir)
  # df_performance_table = performance_table_v2(df_performance_table, get_UMAFall_loader, get_UPFall_loader, src_name, tgt_name, optimal_training_params, inputdir, outputdir)

  time_elapsed = time.time() - start_time
  print('time elapsed:', time_elapsed)
  df_performance_table.loc['time_elapsed'] = time_elapsed

  df_outputdir = outputdir

  print('df_performance_table saved at', df_outputdir)
  df_performance_table.to_csv(df_outputdir+'df_performance_table_optimal.csv', encoding='utf-8')

  display(df_performance_table)

Output hidden; open in https://colab.research.google.com to view.

## hyperparameter testing for 

In [0]:
# # fine-tuning
# tasks_list = [('UMAFall_ankle', 'UPFall_ankle')]

# training_params_list = [
#   {
#     'classes_n': 2,
#     'CV_n': 17,
#     'num_epochs': 10,
#     'channel_n': 4,
#     'batch_size': 16,
#     'learning_rate': 0.01,
#     'extractor_type': 'CNN'}, 
#   {
#     'classes_n': 2,
#     'CV_n': 17,
#     'num_epochs': 10,
#     'channel_n': 4,
#     'batch_size': 4,
#     'learning_rate': 0.01,
#     'extractor_type': 'CNN'}, 
#   {
#     'classes_n': 2,
#     'CV_n': 17,
#     'num_epochs': 10,
#     'channel_n': 4,
#     'batch_size': 64,
#     'learning_rate': 0.01,
#     'extractor_type': 'CNN'}, 
    
#   {
#     'classes_n': 2,
#     'CV_n': 17,
#     'num_epochs': 10,
#     'channel_n': 16,
#     'batch_size': 4,
#     'learning_rate': 0.01,
#     'extractor_type': 'CNN'}, 
#   {
#     'classes_n': 2,
#     'CV_n': 17,
#     'num_epochs': 10,
#     'channel_n': 32,
#     'batch_size': 4,
#     'learning_rate': 0.01,
#     'extractor_type': 'CNN'}, 

#   {
#     'classes_n': 2,
#     'CV_n': 17,
#     'num_epochs': 10,
#     'channel_n': 4,
#     'batch_size': 4,
#     'learning_rate': 0.001,
#     'extractor_type': 'CNN'}, 
#   {
#     'classes_n': 2,
#     'CV_n': 17,
#     'num_epochs': 10,
#     'channel_n': 4,
#     'batch_size': 4,
#     'learning_rate': 0.0001,
#     'extractor_type': 'CNN'}, ]

# for i, training_params in enumerate(training_params_list):
#   start_time = time.time()
#   df_performance_table = pd.DataFrame('', index=['source', 'DANN', 'target', 'domain'], columns=[])

#   for task_item in tasks_list:
#     (src_name, tgt_name) = task_item

#     # inputdir = '/content/drive/My Drive/中研院/data_mic/stage1_preprocessed_18hz/'
#     # outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz/{}_{}/'.format(src_name, tgt_name)

#     inputdir = '/content/drive/My Drive/中研院/data_mic/stage1_preprocessed_18hz_LOO/'
#     outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz_LOO/{}_{}/'.format(src_name, tgt_name)
#     if not os.path.exists(outputdir):
#         os.makedirs(outputdir)
#     print('outputdir for stage2 output:', outputdir)

#     df_performance_table = performance_table(df_performance_table, src_name, tgt_name, training_params, inputdir, outputdir)
#     # display(df_performance_table)

#   time_elapsed = time.time() - start_time
#   print('time elapsed:', time_elapsed)
#   df_performance_table.loc['time_elapsed'] = time_elapsed
#   display(df_performance_table)

#   df_outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz_LOO/hyperparameter_ft/{}_{}/'.format(src_name, tgt_name)
#   if not os.path.exists(df_outputdir):
#     os.makedirs(df_outputdir)
#   df_performance_table.to_csv(df_outputdir+'df_performance_table_i{}.csv'.format(i), encoding='utf-8')


In [0]:
# df_performance_table = pd.DataFrame(0, index=['source', 'DANN', 'target', 'domain'], columns=[])
# tasks_list = [('UMAFall_waist', 'UPFall_belt'), ('UMAFall_ankle', 'UPFall_ankle'), ('UMAFall_wrist', 'UPFall_wrist'), ('UMAFall_leg', 'UPFall_rightpocket'), ('UMAFall_all', 'UPFall_all')]

# optimal_training_params = {
#     'classes_n': 2,
#     'CV_n': 5,
#     'num_epochs': 3,
#     'channel_n': 32,
#     'batch_size': 1,
#     'learning_rate': 0.01}

# for task_item in tasks_list:
#   (src_name, tgt_name) = task_item

#   # inputdir = '/content/drive/My Drive/中研院/data_mic/stage1_preprocessed/'
#   # outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput/{}_{}/'.format(src_name, tgt_name)
#   inputdir = '/content/drive/My Drive/中研院/data_mic/stage1_preprocessed_18hz/'
#   outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz/{}_{}/'.format(src_name, tgt_name)
#   if not os.path.exists(outputdir):
#       os.makedirs(outputdir)
#   print('outputdir for stage2 output:', outputdir)

#   performance_table(src_name, tgt_name, optimal_training_params, df_performance_table, inputdir, outputdir)
#   display(df_performance_table)

# "remove one sensor at a time" experiment

In [0]:
df_performance_table = pd.DataFrame(0, index=['source', 'DANN', 'target', 'domain'], columns=[])
tasks_list = [('UMAFall_allbutwrist', 'UPFall_allbutwrist'), ('UMAFall_allbutwaist', 'UPFall_allbutwaist'), ('UMAFall_allbutleg', 'UPFall_allbutleg'), ('UMAFall_allbutankle', 'UPFall_allbutankle'), ('UMAFall_allbutfifth', 'UPFall_allbutfifth')]
# tasks_list = [('UMAFall_allbutwrist', 'UPFall_allbutwrist'), ('UMAFall_ankle', 'UPFall_ankle'), ('UMAFall_wrist', 'UPFall_wrist'), ('UMAFall_leg', 'UPFall_rightpocket'), ('UMAFall_all', 'UPFall_all')]

optimal_training_params = {
    'classes_n': 2,
    'CV_n': 5,
    'num_epochs': 3,
    'channel_n': 32,
    'batch_size': 1,
    'learning_rate': 0.01}

for task_item in tasks_list:
  (src_name, tgt_name) = task_item

  # inputdir = '/content/drive/My Drive/中研院/data_mic/stage1_preprocessed/'
  # outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput/{}_{}/'.format(src_name, tgt_name)
  inputdir = '/content/drive/My Drive/中研院/data_mic/stage1_preprocessed_18hz/'
  outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz/{}_{}/'.format(src_name, tgt_name)
  if not os.path.exists(outputdir):
      os.makedirs(outputdir)
  print('outputdir for stage2 output:', outputdir)

  performance_table(src_name, tgt_name, optimal_training_params, df_performance_table, inputdir, outputdir)
  display(df_performance_table)

# Repeat 10 times experiment

In [0]:
# for i in range(0,10):
for i in range(1,5):
  start_time = time.time()

  df_performance_table = pd.DataFrame(0, index=['source', 'DANN', 'target', 'domain'], columns=[])
  # tasks_list = [('UMAFall_waist', 'UPFall_belt'), ('UMAFall_ankle', 'UPFall_ankle'), ('UMAFall_wrist', 'UPFall_wrist'), ('UMAFall_leg', 'UPFall_rightpocket'), ('UMAFall_all', 'UPFall_all')]
  tasks_list = [('UMAFall_waist', 'UPFall_belt'), ('UMAFall_ankle', 'UPFall_ankle'), ('UMAFall_wrist', 'UPFall_wrist'), ('UMAFall_leg', 'UPFall_rightpocket')]

  # optimal_training_params = {
  #     'classes_n': 2,
  #     'CV_n': 5,
  #     'num_epochs': 3,
  #     'channel_n': 32,
  #     'batch_size': 1,
  #     'learning_rate': 0.01}

  optimal_training_params = {
      'classes_n': 2,
      'CV_n': 10,
      'num_epochs': 10,
      'channel_n': 4,
      'batch_size': 4,
      'learning_rate': 0.01,
      'extractor_type': 'CNN'}

  for task_item in tasks_list:
    (src_name, tgt_name) = task_item


    # inputdir = '/content/drive/My Drive/中研院/data_mic/stage1_preprocessed_18hz/'
    # outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz/{}_{}/'.format(src_name, tgt_name)
    inputdir = '/content/drive/My Drive/中研院/data_mic/stage1_preprocessed_18hz_LOO/'
    outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz_LOO/{}_{}/'.format(src_name, tgt_name)
    if not os.path.exists(outputdir):
        os.makedirs(outputdir)
    print('outputdir for stage2 output:', outputdir)

    # performance_table(src_name, tgt_name, optimal_training_params, df_performance_table, inputdir, outputdir)
    df_performance_table = performance_table(df_performance_table, src_name, tgt_name, optimal_training_params, inputdir, outputdir)
    display(df_performance_table)

  # df_outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz/repetitive_results/'
  df_outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz_LOO/repetitive_results/'
  if not os.path.exists(df_outputdir):
    os.makedirs(df_outputdir)
  df_performance_table.to_csv(df_outputdir+'df_performance_table_trial{}.csv'.format(i), encoding='utf-8')

  elapsed_time = time.time() - start_time
  print('time elapsed:', elapsed_time)


In [0]:
# for i in range(0,10):
# # for i in range(0,1):
#   start_time = time.time()

#   df_performance_table = pd.DataFrame(0, index=['source', 'DANN', 'target', 'domain'], columns=[])
#   tasks_list = [('UMAFall_waist', 'UPFall_belt'), ('UMAFall_ankle', 'UPFall_ankle'), ('UMAFall_wrist', 'UPFall_wrist'), ('UMAFall_leg', 'UPFall_rightpocket'), ('UMAFall_all', 'UPFall_all')]
#   # tasks_list = [('UMAFall_waist', 'UPFall_belt'), ('UMAFall_ankle', 'UPFall_ankle'), ('UMAFall_wrist', 'UPFall_wrist'), ('UMAFall_leg', 'UPFall_rightpocket')]

#   # optimal_training_params = {
#   #     'classes_n': 2,
#   #     'CV_n': 5,
#   #     'num_epochs': 3,
#   #     'channel_n': 32,
#   #     'batch_size': 1,
#   #     'learning_rate': 0.01}

#   optimal_training_params = {
#       'classes_n': 2,
#       'CV_n': 10,
#       'num_epochs': 5,
#       'channel_n': 4,
#       'batch_size': 4,
#       'learning_rate': 0.01,
#       'extractor_type': 'CNN'}

#   for task_item in tasks_list:
#     (src_name, tgt_name) = task_item


#     # inputdir = '/content/drive/My Drive/中研院/data_mic/stage1_preprocessed_18hz/'
#     # outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz/{}_{}/'.format(src_name, tgt_name)
#     inputdir = '/content/drive/My Drive/中研院/data_mic/stage1_preprocessed_18hz_LOO/'
#     outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz_LOO/{}_{}/'.format(src_name, tgt_name)
#     if not os.path.exists(outputdir):
#         os.makedirs(outputdir)
#     print('outputdir for stage2 output:', outputdir)

#     # performance_table(src_name, tgt_name, optimal_training_params, df_performance_table, inputdir, outputdir)
#     df_performance_table = performance_table(df_performance_table, src_name, tgt_name, optimal_training_params, inputdir, outputdir)
#     display(df_performance_table)

#   # df_outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz/repetitive_results/'
#   df_outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz_LOO/repetitive_results/'
#   if not os.path.exists(df_outputdir):
#     os.makedirs(df_outputdir)
#   df_performance_table.to_csv(df_outputdir+'df_performance_table_trial{}.csv'.format(i), encoding='utf-8')

#   elapsed_time = time.time() - start_time
#   print('time elapsed:', elapsed_time)


# fill in the rest of performance table

In [0]:
# for i in range(0,10):
df_performance_table = pd.DataFrame(0, index=['source', 'DANN', 'target', 'domain'], columns=[])
tasks_list = [('UPFall_belt', 'UMAFall_waist'), ('UPFall_ankle', 'UMAFall_ankle'), ('UPFall_wrist', 'UMAFall_wrist'), ('UPFall_rightpocket', 'UMAFall_leg'), ('UPFall_neck', 'UMAFall_chest'), ('UMAFall_all', 'UPFall_all')]

optimal_training_params = {
    'classes_n': 2,
    'CV_n': 5,
    'num_epochs': 3,
    'channel_n': 32,
    'batch_size': 1,
    'learning_rate': 0.01}

for task_item in tasks_list:
  (src_name, tgt_name) = task_item

  inputdir = '/content/drive/My Drive/中研院/data_mic/stage1_preprocessed_18hz/'
  outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz/{}_{}/'.format(src_name, tgt_name)
  if not os.path.exists(outputdir):
      os.makedirs(outputdir)
  print('outputdir for stage2 output:', outputdir)

  performance_table(src_name, tgt_name, optimal_training_params, df_performance_table, inputdir, outputdir)
  display(df_performance_table)

  # df_outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput_18hz/repetitive_results/'
  # if not os.path.exists(df_outputdir):
  #   os.makedirs(df_outputdir)
  # df_performance_table.to_csv(df_outputdir+'df_performance_table_trial{}.csv'.format(i), encoding='utf-8')



# Feature space clustering